# IUM-zespół 5 wariant 02
Marcin Jarczewski
Miłosz Mizak

# Wstęp

W ramach drugiego etapu projektu stworzyliśmy:
- model bazowy
- model zaawansowany (docelowy)
- API służące do interakcji z modelami

## Klasy

Oba modele korzystają z klas reprezentujących piosenkę oraz składankę. Klasa `Track` jest bardzo prosta i zawiera tylko trzy pola: `track_id`, `popularity` i `group`. Klasa `Playlist` jest już nieco bardziej skomplikowana. Zawiera ona funkcje, które aktualizują informacje na temat składanki za każdym razem, gdy dodawana jest nowa piosenka — jej wielkość, statystyki na temat popularności piosenek w składance, a także łączną długość wszystkich piosenek.

## Model bazowy

Model bazowy działa w bardzo prosty sposób: tworzy składanki z tych gatunków muzycznych, które posiadają więcej niż X piosenek, gdzie X jest parametrem modelu. Domyślnie jest to ustawione na wartość 500 piosenek. Piosenki z danego gatunku są sortowane według ich wskaźnika `popularity`, a następnie w składance jest umieszczane Y najpopularniejszych utworów, gdzie Y to liczba z zakresu [10-20]. Po wygenerowaniu składanek model wybiera te, które mają największą średnią popularność. Ilość tych składanek także jest parametrem modelu.

## Model zaawansowany

Główną funkcją modelu zaawansowanego jest grupowanie piosenek. Użyliśmy do tego gotowego modelu KMeans, dostępnego w bibliotece scikit-learn. Model działa w następujący sposób:
- model przyporządkowuje każdą piosenkę ze zbioru danych do określonej grupy. Testowaliśmy różne ilości grup i przy obecnym rozmiarze zbioru danych 10 grup wydaje się rozsądnym wyborem. Większa ich ilość generowała patologie — przy 30 grupach piosenki o dokładnie takich samych parametrach były przyporządkowywane do zupełnie innych grup.
- piosenki zostają posortowane ze względu na grupy, a następnie z danych wybierane są te piosenki, które przekraczają określony próg popularności (parametr `popularity`) - jako domyślna wartość jest to 95. 
- popularne piosenki znajdują się teraz w odpowiednich grupach, więc inne piosenki powinny być do nich podobne — wobec tego z grup w którch znajdują się te piosenki, losujemy daną ilość piosenek.
- wylosowane piosenki dodajemy do składanek, które potem są wszystkie razem zwracane jako wynik działania modelu.

## Serwis

Do uruchamiania modeli zostało oddelegowane API. API posiada dwa endpointy jeden z nich: `/predict/model/{model_type}`, gdzie jako `model_type` można użyć zarówno modelu bazowego, jak i zaawansowanego. Liczba generowanych składanek zawsze wynosi 5, natomiast istnieje możliwość sprecyzowania ilości piosenek w składance poprzez modyfikację pliku `config.py`.

Drugi endpoint służy do przeprowadzenia testów A/B: `/playlists/{user_id}`, na podstawie reszty z dzielenia przez dwa jest przypisywany model, który zostanie użyty do wygenerowania składanki.

Aby przetestować działanie mikroserwisu, należy najpierw pobrać wszystkie niezbędne biblioteki używając komendy `pip -r requirements.txt`, a następnie uruchomić `python src/service.py`. Na adresie `0.0.0.0:8000` zostanie uruchomiony serwis.

![Api](./figures/api.png)
![Api - a/b endpoint](./figures/endpoint-a-b.png)

Dodatkowo można uruchomić nasz serwis przy wykorzystaniu Dockera, korzystając z poleceń:
1. `docker build -t ium-service-image .`
2. `docker run -it --rm --name ium-service -p 8000:8000 ium-service-image`

Analogicznie serwis będzie dostępny pod adresem `0.0.0.0:8000`.

## Obserwacje

### Porównanie modeli

Postanowiliśmy przeprowadzić wstępną analizę porównawczą modeli. Skrypt `compare.py` uruchamia modele, generując dwie grupy playlist. Generowanie playlist odbywa się 10 razy, a wyniki są agregowane, tak aby porównanie było możliwe niezależne od warunków losowych. Z każdej playlisty pobieramy następujące dane:
- rozmiar playlisty
- popularność najbardziej i najmniej popularnego utworu
- średnia popularność playlisty (średnia ze wszystkich popularności piosenek)
- listę utworów
- retencję - wskaźnik liczony na podstawie danych z pliku `track_user_stats.jsonl`.

Wyniki porównania modeli widać poniżej.

In [ ]:
{
  'base': 
    {
      'size': 10.0, 
      'min_track_popularity': 87.20000000000002, 
      'max_track_popularity': 96.79999999999998, 
      'mean_popularity': 90.84, 
      'retention': 80.87695086524332
    }, 
  'advanced':
    {
      'size': 10.0,
      'min_track_popularity': 52.15500000000001,
      'max_track_popularity': 74.32499999999999,
      'mean_popularity': 61.358999999999995,
      'retention': 76.36434844287751
    }
}

Opierając się na danych sprzed wprowadzenia modelu zaawansowanego, widzimy, że retencja modelu bazowego jest wyższa o około 4 p.p. Nie jest to jednak zbyt duża zmiana, biorąc pod uwagę popularność piosenek. Model bazowy wybiera jedynie najpopularniejsze piosenki, zaś zaawansowany nie posiada takich ograniczeń. Skoro retencja modelu zaawansowanego jest tylko nieco mniejsza, to istnieje duża szansa, że w testach online jego playlisty przyciągną do siebie użytkowników na dłużej.

### Kryterium sukcesu

Niestety, na podstawie jedynie danych offline nie jesteśmy w stanie ocenić, czy kryterium sukcesu zostało spełnione. Konieczne są testy w środowisku produkcyjnym, gdyż sama obecność nowych playlist ma szansę wygenerować większą retencję na piosenkach w tych playlistach.